# Minesweeper AI

In [79]:
import math
import numpy as np
import copy
import tensorflow as tf

In [80]:
class MineSweeper:
    def __init__(self, height = 9, width = 9, mines = 9):
        self.height = height
        self.width = width
        self.num_mines = mines
        self.reset()
        
    def reset(self):
        self.board = np.array([[' ' for i in range(self.width+2)] for j in range(self.height+2)])
        self.done = False
        self.reward = 0
        self.moves = [i for i in range(self.height * self.width)]
        self.mines = []
    
    # Find number of mines adjacent to current square
    def find_mines(self, row, col):
        
        # Return -1 if we hit a mine
        if row*self.width + col in self.mines:
            return -1
        
        num_mines = 0
        for r_offset in [-1, 0, 1]:
            for c_offset in [-1, 0, 1]:
                if (row + r_offset) * self.width + (col + c_offset) in self.mines:
                    num_mines += 1
        
        return num_mines
        
    def step(self, move):
        # Setup
        if move not in self.moves:
            move = self.sample()

        self.moves.remove(move) # Remove move if already stepped
        
        # Make random mine positions after the first move
        if len(self.moves) == self.height*self.width-1:
            self.mines = np.random.choice(self.moves, self.num_mines, replace = False)
    
        row, col = move//self.width, move%self.width
        num_mines = self.find_mines(row, col)
        
        if num_mines == -1:
            self.board[row+1, col+1] = '*'
            self.reward = 0
            self.done = True
            
        # Recursive call opens adjacent cells if they are 0
        else:
            self.board[row+1, col+1] = str(num_mines)
            # Check for 0s and fill in board
            if num_mines == 0:
                for r_offset in [-1, 0, 1]:
                    for c_offset in [-1, 0, 1]:
                        if (r_offset, c_offset) == (0, 0): continue
                        if row+r_offset+1 in [0, self.height+1] or col+c_offset+1 in [0, self.width+1]: continue
                        if (row+r_offset)*self.width + (col+c_offset) in self.moves:
                            self.step((row+r_offset)*self.width + (col+c_offset))
            
            # Check whether the gameboard is solved
            if len(self.moves) == self.num_mines:
                self.reward = 1
                self.done = True
                
        return
            
    def sample(self):
        return np.random.choice(self.moves)
    
    # One hot enconding
    def get_network_state(self):
        miniboard = self.board[1:-1, 1:-1]
        myboard = [np.array(miniboard == ' ', dtype = int)]
        
        for i in range(9):
            myboard.append(np.array(miniboard==str(i), dtype = int))
        return tf.convert_to_tensor(myboard)
        

        
    def render(self):
        for rows in self.board[1:-1]:
            print('|'.join(rows))
    
    def render_with_mines(self):
        self.mineboard = copy.deepcopy(self.board)
        for mine in self.mines:
            self.mineboard[mine//self.height + 1, mine % self.height + 1] = '*'
        for rows in self.mineboard[1:-1]:
            print('|'.join(rows))

In [81]:
env = MineSweeper(9, 9, 9)

In [82]:
env.render()

 | | | | | | | | | | 
 | | | | | | | | | | 
 | | | | | | | | | | 
 | | | | | | | | | | 
 | | | | | | | | | | 
 | | | | | | | | | | 
 | | | | | | | | | | 
 | | | | | | | | | | 
 | | | | | | | | | | 


In [83]:
# Randomly choose from available moves/squares
def RandomAgent(state, moves):
    return np.random.choice(moves)

In [84]:
def neural_network(state, moves):
    probs = nn_model(tf.reshape(env.get_network_state(), (1, 9, 9, 10))).numpy().flatten()
    probs[moves] += 1
    return tf.argmax(probs)

In [85]:
def play(env, agent = RandomAgent, episodes=100, verbose = False):
    total_reward = 0
    for game in range(episodes):
        env.reset()
        while not env.done:
            move = agent(env.get_network_state(), env.moves)
            env.step(move)
            
            if verbose:
                print('Move: ', move//9, move%9)
                env.render()
                print()
        
        total_reward += env.reward

    print(f'Number of wins in {episodes} games: {total_reward}')

In [86]:
play(env = MineSweeper(9, 9, 9), agent = RandomAgent, episodes = 100)

Number of wins in 100 games: 0


# Neural Network Approach to Solving Minesweeper

Input: Tensor state of the minesweeper board

Output: Probabilities of what cells to choose/expand

Loss: Crossentropy loss

In [87]:
from tensorflow import keras
from tensorflow.keras import layers

In [88]:
MINES = 9
HEIGHT = 9
WIDTH = 9

In [89]:
env = MineSweeper(HEIGHT, WIDTH, MINES)
nn_model = keras.Sequential(
    [
        keras.Input(shape=(HEIGHT, WIDTH, 10)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.Flatten(),
        layers.Dense(HEIGHT*WIDTH, activation="softmax"),
    ]
)

In [90]:
nn_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 7, 7, 32)          2912      
                                                                 
 conv2d_11 (Conv2D)          (None, 5, 5, 64)          18496     
                                                                 
 flatten_5 (Flatten)         (None, 1600)              0         
                                                                 
 dense_5 (Dense)             (None, 81)                129681    
                                                                 
Total params: 151089 (590.19 KB)
Trainable params: 151089 (590.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [109]:
# Acquire training data for network
def get_samples(model, samples = 100):
    x, y = [], []
    env = MineSweeper(HEIGHT, WIDTH, MINES)
    for game in range(samples):
        env.reset()
        while not env.done:
            current_state = tf.transpose(env.get_network_state(), perm = [1, 2, 0])
            move = RandomAgent(current_state, env.moves)
            env.step(move)
            # Exclude samples that fail
            if env.done and env.reward == 0:
                continue
            else:
                x.append(current_state)
                y.append([i if i == move else 0 for i in range(HEIGHT*WIDTH)])
            
    return tf.convert_to_tensor(x), tf.convert_to_tensor(y)

In [110]:
x_train, y_train = get_samples(nn_model, samples = 500)

In [111]:
x_train.shape, y_train.shape

(TensorShape([2071, 9, 9, 10]), TensorShape([2071, 81]))

In [112]:
# Compile and fit model
nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
nn_model.fit(x_train, y_train, batch_size=16, epochs=100, validation_split=0.1, verbose=1)

Epoch 1/100
117/117 [==============================] - 2s 11ms/step - loss: 24267585536.0000 - accuracy: 0.0134 - val_loss: 21189799936.0000 - val_accuracy: 0.0096
Epoch 2/100
117/117 [==============================] - 1s 10ms/step - loss: 24144965632.0000 - accuracy: 0.0129 - val_loss: 20984193024.0000 - val_accuracy: 0.0337
Epoch 3/100
117/117 [==============================] - 1s 9ms/step - loss: 24061003776.0000 - accuracy: 0.0113 - val_loss: 20915232768.0000 - val_accuracy: 0.0048
Epoch 4/100
117/117 [==============================] - 1s 9ms/step - loss: 23848196096.0000 - accuracy: 0.0166 - val_loss: 20744988672.0000 - val_accuracy: 0.0337
Epoch 5/100
117/117 [==============================] - 1s 10ms/step - loss: 23703889920.0000 - accuracy: 0.0172 - val_loss: 20536856576.0000 - val_accuracy: 0.0096
Epoch 6/100
117/117 [==============================] - 1s 9ms/step - loss: 23445116928.0000 - accuracy: 0.0107 - val_loss: 20283371520.0000 - val_accuracy: 0.0144
Epoch 7/100
117/117

In [119]:
for i in range(10):
    play(env = MineSweeper(HEIGHT, WIDTH, 3), agent = neural_network, episodes = 100)

Number of wins in 100 games: 31
Number of wins in 100 games: 31
Number of wins in 100 games: 36
Number of wins in 100 games: 28
Number of wins in 100 games: 34
Number of wins in 100 games: 35
Number of wins in 100 games: 28
Number of wins in 100 games: 29
Number of wins in 100 games: 29
Number of wins in 100 games: 33


In [120]:
for i in range(10):
    play(env = MineSweeper(HEIGHT, WIDTH, 3), agent = RandomAgent, episodes = 100)

Number of wins in 100 games: 26
Number of wins in 100 games: 28
Number of wins in 100 games: 26
Number of wins in 100 games: 27
Number of wins in 100 games: 24
Number of wins in 100 games: 28
Number of wins in 100 games: 20
Number of wins in 100 games: 28
Number of wins in 100 games: 22
Number of wins in 100 games: 30
